In [1]:
import tensorflow as tf
from tensorflow import keras

2025-03-10 21:14:12.371391: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-10 21:14:12.386414: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-10 21:14:12.404188: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-10 21:14:12.409459: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-10 21:14:12.422271: I tensorflow/core/platform/cpu_feature_guar

In [4]:
class PlaylistTower(keras.Model):
    '''
    Produced embedding represents the features of a playlist known at query time.
    '''
    def __init__(self, layer_sizes, vocab_dict):
        super().__init__()
        # TODO: build sequential model for each feature here.

        # Feature: pl_name_src
        self.pl_name_src_text_embedding = keras.Sequential([
            keras.layers.TextVectorization(
                vocabulary=vocab_dict['pl_name_src'],
                ngram=2,
                name='pl_name_src_textvectorizor'
            ),
            keras.layers.Embedding(
                input_dim = MAX_TOKENS,
                output_dim = EMBEDDING_DIM,
                name = "pl_name_src_emb_layer",
                mask_zero=False
            ),
            keras.layers.GlobalAveragePooling1D(
                name="pl_name_src_id"
            )
        ], name="pl_name_src_text_embedding")

        # Feature: artist_genres_pl
        self.artist_genres_pl_embedding = tf.keras.Sequential([
            tf.keras.layers.TextVectorization(
                ngrams=2,
                vocabulary=vocab_dict['artist_genres_pl'],
                name="artist_genres_pl_textvectorizor"
            ),
            tf.keras.layers.Embedding(
                input_dim=MAX_TOKENS, 
                output_dim=EMBED_DIM,
                name="artist_genres_pl_emb_layer",
                mask_zero=False
            ),
            tf.keras.layers.Reshape([-1, MAX_PL_LENGTH, EMBED_DIM]),
            tf.keras.layers.GlobalAveragePooling2D(name="artist_genres_pl_2d"),
        ], name="artist_genres_pl_emb_model")
            
    def call(self, data):
        '''
        Defines what happens when the model is called.
        '''
        all_embs = tf.concat([
            # TODO: concatenate output of all features defined above
        ], axis=1)
        # Pass output to dense / cross layers
        if self._cross_layer is not None:
            cross_embs = self._cross_layer(all_embs)
            return self.dense_layers(cross_embs)
        else:
            return self.dense_layers(all_embs)

In [ ]:
class CandidateTrackTower(keras.Model):
    def __init__(self):
        pass
    
    def call(self):
        pass

In [ ]:
class TheTwoTowers(keras.Model):
    def __init__(self, layer_sizes, vocab_dict, parsed_candidate_dataset):
        super().__init__()

        self.query_tower = PlaylistTower(layer_sizes, vocab_dict)

        self.candidate_tower = CandidateTrackTower(layer_sizes, vocab_dict)

        self.task = tf.tasks.Retrieval(
            metrics=tf.metrics.FactorizedTopK(
                candidates=parsed_candidate_dataset.batch(128).map(
                    self.candidate_tower,
                    num_parallel_calls=tf.data.AUTOTUNE
                ).prefetch(tf.data.AUTOTUNE)
            )
        )

    def compute_loss(self, data, training=False):
        
        query_embeddings = self.query_tower(data)
        candidate_embeddings = self.candidate_tower(data)
        
        return self.task(
            query_embeddings,
            candidate_embeddings,
            compute_metrics=not training,
            candidate_ids=data['track_uri_can'],
            compute_batch_metrics=True
        )